# Задача

В рамках данной задачи предстоит научиться получать наилучшее приближение $\sigma\in\mathbb{C}^n$ истинного квантового состояния $\rho\in\mathbb{C}^n$ по некоторой обучающей выборке $X_{train}=\{E_m,y_m\}_{m=0}^{N}$, где $E_m$ - некоторая матрица (называемая оператор, проектор), $y_m$ - некоторое наблюдение (проекция истинного состояния $\rho$ на этот оператор). 

В качестве оптимизируемого функционала предлагается минимизировать следующую функцию:
$$min_\sigma[f(\sigma)=\Sigma_m(Tr(E_m\sigma)-Tr(E_m\rho))^2=\Sigma_m(Tr(E_m\sigma)-y_m)^2]$$
$$\sigma\geq0,Tr(\sigma)=1$$
гдe $\sigma\geq0$ означает положительную полуопределённость обучаемой матрицы $\sigma$ 

# Решение

Поскольку матрица $\sigma$ - положительно полуопределённая, сделаем разложение Холецкого: $\sigma=M^*M$. 

Тогда 
$$Tr(\sigma)=Tr(M^*M)=\Sigma_{a_m\in M} a_m^*a_m=\Sigma_{a_m\in M} |a_m|^2$$

Поскольку для любой положительно полуопределённой матрицы существует разложение Холецкого, и, наоборот, для любой матрицы $M\in\mathbb{C}^n$ матрица $M^*M$ является положительно полуопределённой, мы можем переписать оптимизационную задачу в виде

$$min_M[f(M)=\Sigma_m(Tr(E_mM^*M)-y_m)^2]$$
$$Tr(M^*M)=\Sigma_{a_m\in M}|a_m|^2=1$$

Эту задачу можно было бы решать обычным методом множителей Лагранжа, но можно просто перенормировать $M$ на $\sqrt{Tr(M^*M)}$. Тогда задача сведётся к задаче безусловной оптимизации

$$min_M[f(M)=\Sigma_m(Tr(E_m\frac{M^*M}{Tr(M^*M)})-y_m)^2=\Sigma_m(Tr(E_m\frac{M^*M}{\Sigma_{a_m\in M}|a_m|^2})-y_m)^2]$$

# Код

## Генерация датасета

In [1]:
import sys
import torch
import numpy as np
sys.path.append('..')
import lib

In [2]:
rho = np.array([[1., 0.], [0., 0.]]) # target state
dim = rho.shape[0]

projectors_cnt = 10
measurements_cnt = 100

train_size = projectors_cnt * measurements_cnt
train_X, train_y = lib.generate_dataset(rho, projectors_cnt, measurements_cnt)
train_y = train_y.astype('float64')

## Обучение модели

In [3]:
M = lib.ComplexTensor(lib.simulator.randomMixedState(rho.shape[0]))
M.requires_grad = True
opt = torch.optim.SGD([M], lr=1e-1)
epoches = 10

def trace(tensor):
    return sum([tensor[i:i+1,i:i+1] for i in range(tensor.shape[0])])

In [4]:
for epoch in range(epoches):
    sigma = M.t(conjugate=True).mm(M)
    norm = trace(sigma).real.sum()

    loss = 0
    for E_m,y_m in zip(train_X, train_y.astype('float64')):
        E_m = lib.ComplexTensor(E_m)
        loss += ((trace(E_m.mm(sigma)/norm)-y_m)**2).sum()
    loss /= train_size
    loss.backward()
    opt.step()
    opt.zero_grad()
    print(f'Epoch: {epoch}, Loss: {loss.detach().cpu().numpy()}')

Epoch: 0, Loss: 0.0005043209530413151
Epoch: 1, Loss: 0.0005034722271375358
Epoch: 2, Loss: 0.000502625189255923
Epoch: 3, Loss: 0.0005017802468501031
Epoch: 4, Loss: 0.0005009371670894325
Epoch: 5, Loss: 0.000500095949973911
Epoch: 6, Loss: 0.0004992565955035388
Epoch: 7, Loss: 0.0004984191618859768
Epoch: 8, Loss: 0.0004975837655365467
Epoch: 9, Loss: 0.0004967502900399268


In [5]:
sigma = sigma/norm
sigma = sigma.detach().numpy()[:dim] + 1j*sigma.detach().numpy()[dim:]
sigma

array([[0.6505789 +0.j        , 0.10895082-0.25391927j],
       [0.10895082+0.25391927j, 0.3494211 +0.j        ]], dtype=complex64)

In [6]:
lib.fidelity(rho, sigma)

0.6505789160728455